In [1]:
import math
import json

import rasterio
import rasterio.plot
from rasterio.mask import mask 

from osgeo import gdal
gdal.UseExceptions()
    
import geopandas as gpd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from rasterio.plot import plotting_extent
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

import os

In [2]:
def get_bbox(geom):
    pairs = []

    for i in range(0, len(geom)-1, 2):
        pairs.append((geom[i], geom[i+1]))

    x = []
    y = []
    for i in pairs:
        x.append(i[0])
        y.append(i[1])

    bbox = [min(x), min(y), round(max(x)-min(x),2), round(max(y)-min(y),2)]
    
    return bbox

### Abrir arquivos

- Ortomosaico
- Grade
- Vetor das copas

In [3]:
img = rasterio.open('../IMGS/odm_orthophoto.tif')

grid = gpd.read_file('../VECTOR/grid_from_img.geojson').set_index('id')
grid_2 = gpd.read_file('../VECTOR/grid_from_img-2.geojson').set_index('id')

copas = gpd.read_file('../VECTOR/copas.geojson')

In [4]:
info = {
        "description": "Felipe Sa 2021 - HLB",
        "url": "http://siteaqui.com",
        "version": "1.0",
        "year": 2021,
        "contributor": "Felipe Sa",
        "date_created": "2021/01/01"
        }

licenses = [
                {
                "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/",
                "id": 1,
                "name": "Attribution-NonCommercial-ShareAlike License"
                },
                {
                "url": "http://creativecommons.org/licenses/by-nc/2.0/",
                "id": 2,
                "name": "Attribution-NonCommercial License"
                }
            ]

In [6]:
files = [int(x.split('.')[0]) for x in os.listdir('../Mask_RCNN/datasets/canopy/train/images')]
files.sort()

images = []

for i in files:
    
    img = rasterio.open('../IMGS/CORTES/corte_{}.tif'.format(i))
    
    
    images.append({ "license": 1,
                    "file_name": "{:04}.tif".format(i),
                    "coco_url": "empty",
                    "height": img.height,
                    "width": img.width,
                    "date_captured": "2020-01-01 00:00:00",
                    "flickr_url": "empty",
                    "id": i
                    })

In [8]:
categories = [{"supercategory": "canopy","id": 1,"name": "hamlin"}]

In [9]:
annotations = []

for i in files:
    
    copas = gpd.read_file('../VECTOR/CORTES/EDITADOS/corte_{}.geojson'.format(i))
    
    for row in copas.index:
        
        geom_px = [float(x) for x in copas.loc[row, 'geometry_image'].split(', ')]
        
        
        annotations.append({"segmentation": [geom_px],
                            "area": copas.loc[row,'geometry'].area/0.0025,
                            "iscrowd": 0,
                            "image_id": i,
                            "bbox": get_bbox(geom_px),
                            "category_id": 1,
                            "id": int(str(i) + '0' + str(row + 1)) # Não definido ainda
                        })
    
        area = copas.loc[row, 'geometry'].area/0.0025

In [10]:
annotation = {
                "info": info,
                "licenses": licenses,
                "images": images,
                "categories": categories,
                "annotations": annotations,# <-- Not in Captions annotations
#                 "segment_info": []  # <-- Only in Panoptic annotations
                }

In [11]:
with open('../Mask_RCNN/datasets/canopy/train/coco_annotations.json', 'w') as outfile:
    json.dump(annotation, outfile)